# Regridding demo datesets using regrid_to_healpix
Before starting this notebook, Create temporaly small data set using the noteoobk 'prep_regrid.ipynb'
To install the regrid_to_healpix tool, do

```pip install "git+https://github.com/EOPF-DGGS/regrid_to_healpix.git"```

In [1]:

import xarray as xr
import numpy as np

In [2]:

small_ds=xr.open_zarr('/Users/todaka/data/RIOMAR/very_small.zarr')
ds = small_ds[["temp", "nav_lon_rho", "nav_lat_rho"]].isel(s_rho=0).stack(point=("y_rho","x_rho")).dropna(dim="point")
ds = small_ds[["temp", "nav_lon_rho", "nav_lat_rho"]].stack(point=("y_rho","x_rho")).dropna(dim="point")
ds




<xarray.Dataset> Size: 379kB
Dimensions:       (s_rho: 40, point: 2059)
Coordinates:
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * point         (point) object 16kB MultiIndex
    nav_lon_rho   (point) float32 8kB dask.array<chunksize=(2059,), meta=np.ndarray>
    nav_lat_rho   (point) float32 8kB dask.array<chunksize=(2059,), meta=np.ndarray>
  * y_rho         (point) float32 8kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (point) float32 8kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    time_counter  datetime64[ns] 8B ...
    time_instant  datetime64[ns] 8B ...
Data variables:
    temp          (s_rho, point) float32 329kB dask.array<chunksize=(40, 2059), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [3]:
import numpy as np
import xarray as xr
import xdggs
from regrid_to_healpix.regrid_to_healpix_bilinear import Set

# 1) arrays
lon = ds["nav_lon_rho"].values.astype(np.float64)
lat = ds["nav_lat_rho"].values.astype(np.float64)

# 2) build operator ONCE
level = 13
device = "cpu"
nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, threshold=0.5, ellipsoid="WGS84")

# 3) cell_ids ONCE
cell_ids = np.asarray(nr.get_cell_ids(), dtype=np.int64)
ncell = int(cell_ids.size)

# 4) function that maps (point,) -> (cell_ids,)
def to_healpix_point(data_1d):
    out = nr.transform(np.asarray(data_1d, dtype=np.float64), lam=0.1)
    return np.asarray(out, dtype=np.float64)

# 5) apply_ufunc (NOTE: output_sizes uses ncell, not cell_ids)
temp_hp = xr.apply_ufunc(
    to_healpix_point,
    ds["temp"],                      # dims: (point,)
    input_core_dims=[["point"]],
    output_core_dims=[["cell_ids"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float64],
    dask_gufunc_kwargs={"output_sizes": {"cell_ids": ncell}},
).assign_coords(cell_ids=("cell_ids", cell_ids))

temp_hp["cell_ids"].attrs.update(
        {
            "grid_name": "healpix",
            "level": int(level),
            "indexing_scheme": "nested",
            "ellipsoid":"WGS84"
        }
    )
temp_hp=temp_hp.pipe(xdggs.decode).dggs.assign_latlon_coords()
temp_hp

/Users/todaka/python/git/regrid_to_healpix/regrid_to_healpix/regrid_to_healpix_bilinear.py:74: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:51.)
  self.M  = M_coo.to_sparse_csr()


<xarray.DataArray 'temp' (s_rho: 40, cell_ids: 3303)> Size: 1MB
dask.array<transpose, shape=(40, 3303), dtype=float64, chunksize=(40, 3303), chunktype=numpy.ndarray>
Coordinates:
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * cell_ids      (cell_ids) int64 26kB 224374613 224374614 ... 225780866
    latitude      (cell_ids) float64 26kB 46.99 46.99 47.0 ... 47.34 47.35 47.35
    longitude     (cell_ids) float64 26kB 357.7 357.6 357.7 ... 357.3 357.3
    time_counter  datetime64[ns] 8B ...
    time_instant  datetime64[ns] 8B ...
Indexes:
    cell_ids  HealpixIndex(level=13, indexing_scheme=nested, kind=pandas)
Attributes:
    long_name:           potential temperature
    units:               Celsius
    online_operation:    instant
    interval_operation:  1 h
    interval_write:      1 h
    cell_methods:        time: point

In [4]:
temp_hp.compute().dggs.explore(alpha=0.3)